EVENTUAL TITLE OF MANUSCRIPT
============================
This notebook is the computational appendix of [arXiv:1605.XXXXX](http://arxiv.abs/1605.XXXXX). We spell out in detail how to obtain the numerical results in the manuscript. We rely on [Trotter-Suzuki-MPI](https://trotter-suzuki-mpi.github.io/), a massively parallel solver for the Gross--Pitaevskii equation, and its [Python wrapper](http://trotter-suzuki-mpi.readthedocs.org/). The calculations use only a single computer, but it can take many hours to finish them.

Preliminaries
-------------
First we import the necessary modules and ensure that we get identical behaviour in Python 2 and 3.

In [ ]:
from __future__ import print_function, division
import trottersuzuki as ts
import numpy as np
import matplotlib.pyplot as plt
import pandas
import time
from scipy.interpolate import spline, InterpolatedUnivariateSpline

We will generate some files to save the results of the calculations. By default, they will go the current directory:

In [ ]:
directory = ""

Precession of two vortices in a two-component BEC
-------------------------------------------------
The following function takes two parameters $l_{\Omega}/r_{12}$ and $R$, and returns $\Omega_{rot}/\Omega$. In the process, it sets up an imaginary time evolution to approximate the ground state of the system, and then performs real time evolution for a number of iterations that is necessary.

In [ ]:
def two_vortices(lOmega_over_r12, radius):
    ###
    #  Definition of variables and functions for the simulation
    ###
    
    ## Fixed variables
    length = 2. * radius                           # physical lenght of the lattice's side
    initial_vortices_distance = 1.0                # physical distance between vortices cores
    n_0 = 4 / (np.pi * length**2)                  # density at the center of the circular box
    if lOmega_over_r12 >= 0.5:                     # set r_{12} / \xi
        r12_over_Xi12 = 10.
    else:
        r12_over_Xi12 = 40.
    
    ### Lattice parameters
    const = 4.                                     # Ensure that one coherence length will be equal to "const" lattice spacings
    dim = int(length * const * r12_over_Xi12) + 1  # linear dimension of the lattice
    print('Linear dimension of the Lattice:', dim)
    
    ### Hamiltonian parameters
    g_a = 10. * r12_over_Xi12**2 / (2. * n_0) # intra-particle coupling (firt component)
    g_b = 10. * r12_over_Xi12**2 / (2. * n_0) # intra-particle coupling (second component)
    g_ab = 0                                  # inter-particle coupling
    omega_rabi = -1. / (lOmega_over_r12**2)   # Rabi coupling (it is negative since the library use a different sign convention)
    
    ### Solver parameters
    delta_t = 2.5e-5  # time for a single step evolution
    
    # Definition of the pinning potential (only for imaginary time evolution)
    gaussian_radius = 0.5/r12_over_Xi12
    gaussian_height = 100000
    
    def external_potential_imag_a(x, y):
        if (x**2 + y**2) >= (length/2.05)**2:
            return 1e10
        elif np.sqrt((x-1/2)**2+y**2) <= gaussian_radius:
            return gaussian_height*np.exp(- ((x-1/2)**2+y**2) / gaussian_radius**2) / (np.pi * gaussian_radius**2)
        else:
            return 0.
    
    def external_potential_imag_b(x, y):
        if (x**2 + y**2) >= (length/2.05)**2:
            return 1e10
        elif np.sqrt((x+1/2)**2+y**2) <= gaussian_radius:
            return gaussian_height*np.exp(- ((x+1/2)**2+y**2) / gaussian_radius**2) / (np.pi * gaussian_radius**2)
        else:
            return 0.
    
    # Definition of the circular well external potential (only for real time evolution)
    def external_potential(x, y):
        if (x**2 + y**2) >= (length/2.1)**2:
            return 1e10
        else:
            return 0.
    
    # Normalized initial state's wave function
    def const_state(x, y):
        return 1./length
    
    # Define the vortex in component 1
    def vortex_a(x, y):
        z = (x-initial_vortices_distance/2.) + 1j*y
        angle = np.angle(z)
        return np.exp(1j * angle)
    
    # Define the vortex in component 2
    def vortex_b(x, y):
        z = (x+initial_vortices_distance/2.) + 1j*y
        angle = np.angle(z)
        return np.exp(1j * angle)
    
    ###
    #  Set up of the simulation
    ###
    
    # Set the geometry of the simulation
    grid = ts.Lattice(dim, length)
    
    # Set the Hamiltonian
    potential_a = ts.Potential(grid)  # Set the pinning potential for the first component 
    potential_a.init_potential(external_potential_imag_a)
    potential_b = ts.Potential(grid)  # Set the pinning potential for the second component
    potential_b.init_potential(external_potential_imag_b)
    hamiltonian = ts.Hamiltonian2Component(grid, potential_a, potential_b, 1., 1.,
                                           g_a, g_ab, g_b, omega_rabi)
    
    # Set the initial state
    state_a = ts.State(grid)        # Initialize the state in the first component
    state_a.init_state(const_state)
    state_b = ts.State(grid)        # Initialize the state in the second component
    state_b.init_state(const_state)
    
    # Imprint the vortices on the corresponding components
    state_a.imprint(vortex_a)
    state_b.imprint(vortex_b)
    
    # Initialize the solver
    solver = ts.Solver(grid, state_a, state_b, hamiltonian, delta_t)
    
    ###
    #  Set up imaginary time evolution
    ###
    
    # Calculate initial vortices distance
    coord_a = ts.vortex_position(grid, state_a, length*0.45)
    coord_b = ts.vortex_position(grid, state_b, length*0.45)
    vortex_distance = np.sqrt((coord_b[0]-coord_a[0])**2 + (coord_b[1]-coord_a[1])**2)
    print('Vortices distance before imaginary evolution', vortex_distance)
    
    # Decide how many imaginary iterations to perform
    # NB: the higher the Rabi coupling, the higher the iterations needed to form the domain wall
    iterations = 1000
    if lOmega_over_r12 < .2:
        num = 1
    elif lOmega_over_r12 < .5:
        num = 4
    elif lOmega_over_r12 < 1.:
        num = 15
    else:
        num = 24
    print('Imaginary iterations between two snapshots:', iterations)
    
    # Start imaginary time evolution
    print('Imaginary time evolution started')
    for i in range(num):
        print(i+1, ' over ', num, ' completed')
        solver.evolve(iterations, True)         # Perform imaginary time evolution
    
    print('---Imaginary evolution completed---')
    
    # Calculate vortices distance after imaginary time evolution
    coord_a = ts.vortex_position(grid, state_a, length*0.45)
    coord_b = ts.vortex_position(grid, state_b, length*0.45)
    vortex_distance = np.sqrt((coord_b[0]-coord_a[0])**2 + (coord_b[1]-coord_a[1])**2)

    # Check whether the vortices distance has significantly changed and if true abort
    if np.abs(vortex_distance - initial_vortices_distance) > grid.delta_x:
        print('Vortices distance after imaginary time evolution:', vortex_distance, " THE SIMULATION WILL BE ABORTED")
        return
    else:
        print('Vortices distance has not significantly changed')
        print('Vortices distance after imaginary time evolution:', vortex_distance)
    
    ###
    #  Set up real time evolution
    ###
    
    # Change external potential to the circular wall without the pinning potential
    potential_a.init_potential(external_potential)
    potential_b.init_potential(external_potential)
    solver.update_parameters()    
    
    # Iterations to perform to get a tot_angle
    max_it_real_time = 20
    if lOmega_over_r12 < .5:
        iterations = 30
    elif lOmega_over_r12 < 1.:
        iterations = 8000
    else:
        iterations = 16000
        
    print('Iterations between two snapshots:', iterations)
    print('Number of snapshots:', max_it_real_time)
    
    angles_12 = []   # List of precession angles
    times = []       # List of times
    const_angle = 0. # Angle offset
    
    # Start real time evolution
    print('---Start real time evolution---')
    for cont in range(max_it_real_time):
        print('Snapshot:', cont)
        solver.evolve(iterations)
        
        # Calculate the precession angle of the vector r_12
        coord_a = ts.vortex_position(grid, state_a, length*0.45)  # Get position of the vortex in the first component
        coord_b = ts.vortex_position(grid, state_b, length*0.45)  # Get position of the vortex in the second component
        
        angles = np.angle(coord_a[0] - coord_b[0] + 1j * (coord_a[1] - coord_b[1])) # Calculate precession angle
        
        # Add an offset
        if cont > 2:
            last_angle = angles_12[len(angles_12)-2]
        else:
            last_angle = angles
        if abs(angles + const_angle - last_angle) > np.pi:
            const_angle += 2. * np.pi * np.sign(last_angle)
        
        angles_12.append(angles + const_angle)
        times.append(delta_t * iterations * (cont+1))
        
    ###
    #  Calculate precession frequency
    ###
    
    # fit angles curve
    fit_par = np.polyfit(times, angles_12, 1)  # fit_par[0] is the precession frequency
    
    # Print the results on screen
    print('l_omega/r12: ' + str(lOmega_over_r12))
    print('Mean freq of precession over Rabi coupling:', fit_par[0] / np.abs(omega_rabi))    
    
    return fit_par[0] / np.abs(omega_rabi)  # return \Omega_{rot} / \Omega

We explore the parameter space $l_{\Omega} / r_{12} \in [0.1, 10]$ and $R = 2, 3.5, 5$: 

In [ ]:
interval = np.linspace(-1,1,21)
parameters = [2, 3.5, 5]
df = pandas.DataFrame(index=range(len(interval)*len(parameters)), columns=["length_x", "l_omega_r12", "omega_rot/omega"], dtype=np.float64)
i = 0
time0 = time.time()
for lOmega_over_r12 in np.power(10, interval):
    for radius in parameters:
        time_i = time.time()
        df.iloc[i] = [2*radius, lOmega_over_r12, two_vortices(lOmega_over_r12, radius)]
        print("Execution time:", time.time()-time_i)
        i += 1
print("Total time:", time.time()-time0)

In [ ]:
plt.figure(figsize=(9,6))
colors = {4:"b", 7:"g", 10:"r"}

for radius in parameters:
    length_x = int(2*radius)
    ordered_left = df[(df['length_x'] == length_x) & (df['l_omega_r12'] <= 0.50118723) & (df['l_omega_r12'] != 0.79432823) & (df['l_omega_r12'] != 0.63095734)].sort('l_omega_r12')
    ordered_right = df[(df['length_x'] == length_x) & (df['l_omega_r12'] >= 0.50118723) & (df['l_omega_r12'] != 0.79432823) & (df['l_omega_r12'] != 0.63095734)].sort('l_omega_r12')
    x_l = ordered_left['l_omega_r12']
    y_l = -ordered_left['omega_rot/omega']
    x_r = ordered_right['l_omega_r12']
    y_r = -ordered_right['omega_rot/omega']
    x_rm = x_r.as_matrix()
    y_rm = y_r.as_matrix()
    x_r2 = np.linspace(x_rm.min(), x_rm.max(), 300)
    y_r2 = spline(x_rm, y_rm, x_r2)
    plt.plot(x_l,y_l, "--", dashes = [12,6], label="", color=colors[length_x])
    if length_x == 4:
        plt.plot(x_r2,y_r2, label=('$R/r_{12}=2$'), color=colors[length_x])
    if length_x == 7:
        plt.plot(x_r2,y_r2, label=('$R/r_{12}=3.5$'), color=colors[length_x])
    if length_x == 10:
        plt.plot(x_r2,y_r2, label=('$R/r_{12}=5$'), color=colors[length_x])


# add the strong coupling limit
x = np.linspace(0,2,100)
y = -4*np.sqrt(2)*x/np.pi
plt.plot(x,y,'-.', label="$\mathrm{strong} \, \mathrm{coupling}$");

# set labels and limits
plt.xscale('log')
plt.xlabel("$l_\Omega/r_{12}$")
plt.ylabel("$\Omega_\mathrm{rot}/\Omega$")
plt.xlim(0.1,10)
plt.ylim(-2,4)
plt.yticks([-2,-1,0,1,2,3,4], ['$-2$','$-1$','$0$','$1$','$2$','$3$','$4$'])
plt.xticks([0.1,1,10],['$10^{-1}$','$10^{0}$','$10^{1}$',])
#plt.grid(False)
plt.axhline(0, color='black',linewidth=1)
lgd = plt.legend(bbox_to_anchor=(0, 1), loc=2, borderaxespad=0.);

plt.savefig("precession_frequencies.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')

Plot figure 1b:

In [ ]:
plt.figure(figsize=(9,6))

for radius in parameters:
    length_x = int(2*radius)
    ordered = df[df['length_x'] == length_x].sort('l_omega_r12')
    x = 1. / (ordered['l_omega_r12'])**2
    y = - x * ordered['omega_rot/omega']
    if length_x == 4:
        label='$R/r_{12}=2$'
    if length_x == 7:
        label='$R/r_{12}=3.5$'
    if length_x == 10:
        label='$R/r_{12}=5$'

    plt.plot(x,y, label=label, color=colors(length_x))
    # positions to inter/extrapolate
    xi = np.linspace(0, .25, 10)
    # spline order: 1 linear, 2 quadratic, 3 cubic ... 
    order = 1
    # do inter/extrapolation   (the data must be monotonously increasing)
    data_x = (np.array(x)[::-1])[:7]
    data_y = (np.array(y)[::-1])[:7]
    s = InterpolatedUnivariateSpline(data_x, data_y, k=order)
    yi = s(xi)
    # add the result for a single off-centered vortex in a cylindrical container; see Pethick&Smith, Chap.(9.4), Eq.(9.46)
    b=0.5;
    plt.plot([0], [1 / (((length_x/2)**2 - b**2))],'o', label="", markersize=10, color=colors(length_x))

# set labels and limits
plt.xlabel("$\Omega/\omega_{12}$")
plt.ylabel("$\Omega_\mathrm{rot}/\omega_{12}$")
plt.xlim(-0.02,.5)
plt.xticks([0.,.25,0.5], ['$0$','$0.25$', '0.5'])
plt.ylim(-.8,.4)
plt.yticks([-.6,-.3,0,.3], ['$-0.6$','$-0.3$','$0$','$0.3$']);
#plt.grid(False)
plt.axhline(0, color='black',linewidth=1);
plt.axvline(0, color='black',linewidth=1);

lgd = plt.legend(bbox_to_anchor=(0.65, 1), loc=2, borderaxespad=0.);

plt.savefig("precession_frequencies_2.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')

Transfer of a single vortex between a two-component BEC
-------------------------------------------------------
Gory details here.

Oscillations of the relative density
------------------------------------
Even more gory details here.